<a href="https://colab.research.google.com/github/NonaHelmi/SC/blob/main/antcolony_JobScheduling1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [29]:
import numpy as np

def initialize_pheromone(num_jobs, num_machines):
    return np.ones((num_jobs, num_machines))  # Initial pheromone levels for each machine and job

def calculate_completion_time(schedule, jobs_times):
    # Calculate the total time for each machine based on the jobs assigned
    machine_times = []
    for machine_jobs in schedule:
        total_time = sum(jobs_times[job] for job in machine_jobs)
        machine_times.append(total_time)
    return max(machine_times)  # Total time is the max time among machines

def ant_colony_job_scheduling(jobs_times, num_machines, num_ants=10, num_iterations=100, alpha=1, beta=2):
    num_jobs = len(jobs_times)
    pheromone = initialize_pheromone(num_jobs, num_machines)

    best_schedule = None
    best_time = float('inf')

    for iteration in range(num_iterations):
        all_schedules = []
        all_times = []

        for ant in range(num_ants):
            schedule = [[] for _ in range(num_machines)]  # Create a schedule for each machine
            available_jobs = list(range(num_jobs))

            while available_jobs:
                probabilities = []
                for job in available_jobs:
                    # Compute the probability for each machine for the current job
                    prob = []
                    for machine in range(num_machines):
                        pheromone_value = pheromone[job, machine]
                        time_for_job = jobs_times[job]  # Time duration for the job
                        heuristic_value = 1 / (time_for_job ** beta)  # Inverse of job time
                        prob.append((pheromone_value ** alpha) * (heuristic_value))

                    prob_sum = sum(prob)
                    if prob_sum > 0:
                        normalized_prob = [p / prob_sum for p in prob]
                    else:
                        normalized_prob = [1 / num_machines] * num_machines  # Uniform probability if all pheromones are zero

                    probabilities.append(normalized_prob)

                # Select a job and a machine based on the probabilities
                selected_job = np.random.choice(available_jobs)
                selected_machine = np.random.choice(range(num_machines), p=probabilities[len(available_jobs) - 1])

                # Assign the selected job to the selected machine
                schedule[selected_machine].append(selected_job)
                available_jobs.remove(selected_job)

            total_time = calculate_completion_time(schedule, jobs_times)
            all_schedules.append(schedule)
            all_times.append(total_time)

            # Update pheromone levels based on total time
            for machine in range(num_machines):
                for job in schedule[machine]:
                    pheromone[job, machine] += 1 / total_time  # Increase pheromone based on quality of solution

        # Evaporation of pheromone
        pheromone *= 0.95

        # Update best schedule found
        min_time_idx = np.argmin(all_times)
        if all_times[min_time_idx] < best_time:
            best_time = all_times[min_time_idx]
            best_schedule = all_schedules[min_time_idx]

    return best_schedule, best_time

# Example usage for Job Scheduling with multiple machines
jobs_times = [2, 4, 6, 8]  # Duration of each job
num_machines = 3  # Number of machines
best_schedule, best_time = ant_colony_job_scheduling(jobs_times, num_machines)
print("Best schedule:", best_schedule)
print("Best total time:", best_time)

Best schedule: [[3], [1, 0], [2]]
Best total time: 8
